In [49]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt

In [74]:
df = pd.read_csv('ML_input_1T.csv', index_col = 0, parse_dates = True)
df = df[['151', 'FF_10M_10', 'FF_SENSOR_10', 'FF_10M_STD_10', 'FX_SENSOR_10', 'Q_GLOB_10',
       'QN_GLOB_10', 'QX_GLOB_10', 'SQ_10']]

In [75]:
df.columns

Index(['151', 'FF_10M_10', 'FF_SENSOR_10', 'FF_10M_STD_10', 'FX_SENSOR_10',
       'Q_GLOB_10', 'QN_GLOB_10', 'QX_GLOB_10', 'SQ_10'],
      dtype='object')

In [76]:
coeff_one = pywt.wavedec(df, 'haar', level = 1, axis = 0)
coeff_two = pywt.wavedec(df, 'haar', level = 2, axis = 0)
coeff_three = pywt.wavedec(df, 'haar', level = 3, axis = 0)
coeff_four = pywt.wavedec(df, 'haar', level = 4, axis = 0)

In [77]:
print(len(coeff_four[0]), len(coeff_four[1]), len(coeff_four[2]), len(coeff_four[3]), len(coeff_four[4]), len(df),  )

98640 98640 197280 394560 789120 1578240


In [78]:
limit = 1576800

def upsample_dwt_interpolate(arr, factor):
    frequency = str(factor) + 'ms'
    index = pd.date_range('1/1/2000', periods=len(arr), freq=frequency)
    data = pd.DataFrame(arr, index = index)
    data = data.resample('ms').asfreq()
    data = data.interpolate('linear')
    data = data.values
    data = data[:limit,:]
    return data

def upsample_dwt_levels(df, coeff):
    data = []
    for i in range(len(coeff)):
        factor = len(df) / len(coeff[i])
        upsample = upsample_dwt_interpolate(coeff[i], factor)
        data.append(upsample)
    return data

one = upsample_dwt_levels(df, coeff_one)
two = upsample_dwt_levels(df, coeff_two)
three = upsample_dwt_levels(df, coeff_three)
four = upsample_dwt_levels(df, coeff_four)

In [86]:
one_0 = pd.DataFrame(one[0], columns = df.columns).add_prefix('dwt1_0_')
one_1 = pd.DataFrame(one[1], columns = df.columns).add_prefix('dwt1_1_')
first = [one_0, one_1]
first = pd.concat(first, axis = 1)

two_0 = pd.DataFrame(two[0], columns = df.columns).add_prefix('dwt2_0_')
two_1 = pd.DataFrame(two[1], columns = df.columns).add_prefix('dwt2_1_')
two_2 = pd.DataFrame(two[2], columns = df.columns).add_prefix('dwt2_2_')
second = [two_0, two_1, two_2]
second = pd.concat(second, axis = 1)

three_0 = pd.DataFrame(three[0], columns = df.columns).add_prefix('dwt3_0_')
three_1 = pd.DataFrame(three[1], columns = df.columns).add_prefix('dwt3_1_')
three_2 = pd.DataFrame(three[2], columns = df.columns).add_prefix('dwt3_2_')
three_3 = pd.DataFrame(three[3], columns = df.columns).add_prefix('dwt3_3_')
third = [three_0, three_1, three_2, three_3]
third = pd.concat(third, axis = 1)

four_0 = pd.DataFrame(four[0], columns = df.columns).add_prefix('dwt4_0_')
four_1 = pd.DataFrame(four[1], columns = df.columns).add_prefix('dwt4_1_')
four_2 = pd.DataFrame(four[2], columns = df.columns).add_prefix('dwt4_2_')
four_3 = pd.DataFrame(four[3], columns = df.columns).add_prefix('dwt4_3_')
four_4 = pd.DataFrame(four[4], columns = df.columns).add_prefix('dwt4_4_')
fourth = [four_0, four_1, four_2, four_3, four_4]
fourth = pd.concat(fourth, axis = 1)

original = pd.read_csv('ML_input_1T.csv', index_col = 0, parse_dates = True)
original_cols = original.columns
original = original[:limit].values
original = pd.DataFrame(original, columns = original_cols)
dwt = pd.concat([first, second, third, fourth], axis = 1)
dwt = pd.concat([original, dwt], axis = 1)
dwt.index = df[:limit].index

In [88]:
dwt.to_csv('ML_input_1T_dwt.csv', sep=',')

In [10]:
"""original = pd.read_csv('ML_input_15T.csv', index_col = 0, parse_dates = True)
original_cols = original.columns
print(original.values.shape)"""


(105216, 51)


In [56]:
test = pd.read_csv('test_15.csv')

In [87]:
dwt.isna().sum()

151                    149848
181                    274153
192                    148964
226                    189595
262                    291428
                        ...  
dwt4_4_FX_SENSOR_10        10
dwt4_4_Q_GLOB_10           10
dwt4_4_QN_GLOB_10          10
dwt4_4_QX_GLOB_10          10
dwt4_4_SQ_10               10
Length: 177, dtype: int64

In [36]:
for i in dwt.columns:
    print(i)

151
181
192
226
262
288
317
373
380
532
year_2015
year_2016
year_2017
month_sin
month_cos
day_sin
day_cos
hour_sin
hour_cos
minute_sin
minute_cos
DR_REGENM_10
RI_REGENM_10
U_BOOL_10
T_DRYB_10
TN_10CM_PAST_6H_10
T_DEWP_10
TN_DRYB_10
T_WETB_10
TX_DRYB_10
U_10
P_NAP_MSL_10
VV_10
AH_10
MOR_10
FF_10M_10
DD_10_sin
DD_10_cos
DDN_10_sin
DDN_10_cos
DD_STD_10_sin
DD_STD_10_cos
DDX_10_sin
DDX_10_cos
FF_SENSOR_10
FF_10M_STD_10
FX_SENSOR_10
Q_GLOB_10
QN_GLOB_10
QX_GLOB_10
SQ_10
dwt1_0_151
dwt1_0_FF_10M_10
dwt1_0_FF_SENSOR_10
dwt1_0_FF_10M_STD_10
dwt1_0_FX_SENSOR_10
dwt1_0_Q_GLOB_10
dwt1_0_QN_GLOB_10
dwt1_0_QX_GLOB_10
dwt1_0_SQ_10
dwt1_1_151
dwt1_1_FF_10M_10
dwt1_1_FF_SENSOR_10
dwt1_1_FF_10M_STD_10
dwt1_1_FX_SENSOR_10
dwt1_1_Q_GLOB_10
dwt1_1_QN_GLOB_10
dwt1_1_QX_GLOB_10
dwt1_1_SQ_10
dwt2_0_151
dwt2_0_FF_10M_10
dwt2_0_FF_SENSOR_10
dwt2_0_FF_10M_STD_10
dwt2_0_FX_SENSOR_10
dwt2_0_Q_GLOB_10
dwt2_0_QN_GLOB_10
dwt2_0_QX_GLOB_10
dwt2_0_SQ_10
dwt2_1_151
dwt2_1_FF_10M_10
dwt2_1_FF_SENSOR_10
dwt2_1_FF_10M_STD

In [226]:
list(dwt[dwt.columns[dwt.columns.str.startswith('dwt1', na=False)]].columns)

['dwt1_0_151',
 'dwt1_0_FF_10M_10',
 'dwt1_0_FF_SENSOR_10',
 'dwt1_0_FF_10M_STD_10',
 'dwt1_0_FX_SENSOR_10',
 'dwt1_0_Q_GLOB_10',
 'dwt1_0_QN_GLOB_10',
 'dwt1_0_QX_GLOB_10',
 'dwt1_0_SQ_10',
 'dwt1_1_151',
 'dwt1_1_FF_10M_10',
 'dwt1_1_FF_SENSOR_10',
 'dwt1_1_FF_10M_STD_10',
 'dwt1_1_FX_SENSOR_10',
 'dwt1_1_Q_GLOB_10',
 'dwt1_1_QN_GLOB_10',
 'dwt1_1_QX_GLOB_10',
 'dwt1_1_SQ_10']